In [24]:
import openmeteo_requests

import requests_cache
from retry_requests import retry
import polars as pl
from datetime import date, timedelta

In [11]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    # Lat and lon for O'Hare
	"latitude": 41.978611,
	"longitude": -87.904724,
	"start_date": "2025-01-01",
	"end_date": "2025-03-29",
	"daily": ["temperature_2m_mean", "temperature_2m_max", "temperature_2m_min", "rain_sum", "snowfall_sum", "weather_code"],
	"timezone": "America/Chicago"
}

responses = openmeteo.weather_api(url, params=params)

In [ ]:
# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]

# Process daily data
daily = response.Daily()
daily_temperature_2m_mean = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_max = daily.Variables(1).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(2).ValuesAsNumpy()
daily_rain_sum = daily.Variables(3).ValuesAsNumpy()
daily_snowfall_sum = daily.Variables(4).ValuesAsNumpy()
daily_weather_code = daily.Variables(5).ValuesAsNumpy()

daily_data = {"date": pl.date_range(
	start=date(2025, 1, 1),
	end=date(2025, 3, 29),
	interval=timedelta(days=1),
	closed="left",
    eager=True
)}

In [27]:
daily_data

{'date': shape: (87,)
 Series: 'literal' [date]
 [
 	2025-01-01
 	2025-01-02
 	2025-01-03
 	2025-01-04
 	2025-01-05
 	…
 	2025-03-24
 	2025-03-25
 	2025-03-26
 	2025-03-27
 	2025-03-28
 ]}